In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler

In [2]:
df = pd.read_csv("train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.compose import ColumnTransformer

In [4]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
df.drop(columns=["Cabin",'PassengerId','Name','Ticket'],axis=1,inplace=True)

In [6]:
df.isna().sum()
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [7]:
trf1 = ColumnTransformer([
    ('imputed_age',SimpleImputer(),[3]),
    ('imputed_embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')


In [8]:
df.isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [9]:
df.dtypes

Survived      int64
Pclass        int64
Sex          object
Age         float64
SibSp         int64
Parch         int64
Fare        float64
Embarked     object
dtype: object

In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=["Survived"],axis=1),df["Survived"],train_size = 0.25,random_state = 0)

In [11]:
# ohe embarked , sex , 

trf2 = ColumnTransformer([
('ohe_sex',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[2]),
('ohe_embarked',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[6])

],remainder='passthrough')

In [12]:
trf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])

In [13]:
pipe = Pipeline([
('trf1',trf1),
('trf2',trf2),
('trf3',trf3)
])

In [14]:
pipe.fit(X_train,y_train)

,steps,"[('trf1', ...), ('trf2', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('imputed_age', ...), ('imputed_embarked', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [15]:
y_pred = pipe.transform(X_test)

In [16]:
y_pred[:]


array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], shape=(669, 10))

In [17]:
y_test

495    0
648    0
278    0
31     1
255    1
      ..
747    1
663    0
217    0
539    1
736    0
Name: Survived, Length: 669, dtype: int64

In [18]:
if len(y_pred.shape) == 2:
    y_pred = np.argmax(y_pred, axis=1)
    print(1)

# Do the same for y_test if needed
if len(y_test.shape) == 2:
    y_test = np.argmax(y_test, axis=1)
    print(2)

from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

1
0.19282511210762332


In [20]:
# dont go to the acuuracy score it is just to show how easy life becomes using the column transformer and pipelines